# TouchFS Examples

This notebook demonstrates the usage of TouchFS (LLM-powered Memory Filesystem), a FUSE-based memory filesystem backed by JSON that can generate filesystem structures using OpenAI's GPT models.

In [ ]:
from dotenv import load_dotenv
from touchfs.core.operations import Memory
from touchfs.content.generator import generate_filesystem

# Load environment variables
load_dotenv()